In [1]:
import cv2
import PyMBVCore as core
import PyMBVDecoding as dec
import PyMBVRendering as ren
import PyMBVOpenMesh as mbvom
import PyMBVAcquisition as acq
import PyMBVParticleFilter as mpf
import PyModel3dTracker as htpf
import PythonModel3dTracker.PythonModelTracker.LandmarksGrabber as ldm
import BlenderMBVLib.BlenderMBVConnection as bmc
import BlenderMBVLib.BlenderMBVConversions as blconv
import PythonModel3dTracker.PythonModelTracker.PFHelpers.ModelTrackingResults as mtr
import PythonModel3dTracker.PythonModelTracker.Paths as paths
import PythonModel3dTracker.PythonModelTracker.ModelTrackingGui as mtg
import BlenderMBVLib.RenderingUtils as ru
import PythonModel3dTracker.PythonModelTracker.DatasetInfo as dsi
import os
import PythonModel3dTracker.PythonModelTracker.DepthMapUtils as dmu
import copy
import numpy as np

-c:7: RuntimeWarning: to-Python converter for boost::function<void (cv::Mat_<double>&)> already registered; second conversion method ignored.
-c:8: RuntimeWarning: to-Python converter for boost::shared_ptr<MBV::HandTracking::ObjectiveResultsCombination> already registered; second conversion method ignored.


-c:8: RuntimeWarning: to-Python converter for std::__cxx11::list<unsigned long, std::allocator<unsigned long> > already registered; second conversion method ignored.


In [6]:
def get_point_idx(mask,p):
    idx = 0    
    for y in range(p[1]+1):
        if y<p[1]:
            x_items = rgb.shape[1]
        else: x_items = p[0]+1
        for x in range(x_items):
            if mask[y][x]: idx +=1
    return idx


def onMouseGetPoint(event, x, y, s, user_data):
    if event == 1:
        depth = user_data[0]
        rgb = user_data[1]
        disp_depth = user_data[2]
        points = user_data[3]
        print('event:{0}, (x,y): ({1},{2})'.format(event,x,y))
        cv2.rectangle(rgb, (x-1, y-1), (x+1,y+1), (255,0,0))
        points.append([x, y, depth[y][x]])
        print('p2d(x,y,d):',x,y,depth[y][x])
        disp_depth[y][x]=60000
        cv2.imshow('rgb',rgb)
        cv2.imshow('depth',disp_depth)
        cv2.waitKey(1)
        
        
def get3dPoints(points2d,camera,depth):
        points = []
        for p in points2d:
            x = p[0]
            y = p[1]
            mbv_points = core.Vector3fStorage()
            point_mask = camera.PointCloud_fromDepthMap(depth, mbv_points)
            idx = get_point_idx(point_mask,(x,y))
            p = mbv_points[idx]
            p_np = np.array([p.x,p.y,p.z])
            print('3d point:',p_np )
            points.append(p_np)
        return points

In [3]:
params_ds = dsi.DatasetInfo()
params_ds.generate('checkerboard')
grabber_auto = htpf.AutoGrabber.create(str(params_ds.format),
                                       core.StringVector([str(s) for s in params_ds.stream_filename]),
                                       str(params_ds.calib_filename))

In [4]:
frame = 0
grabber_auto.seek(frame)
images, calibs = grabber_auto.grab()

depth = images[0]
rgb = images[1]
disp_depth = copy.deepcopy(depth)
cv2.imshow('depth',disp_depth)
cv2.imshow('rgb',rgb)
points2d = []
cv2.setMouseCallback('rgb', onMouseGetPoint, (depth,rgb,disp_depth,points2d)) 
cv2.waitKey(0)
print('points2d:', points2d)

event:1, (x,y): (258,283)
('p2d(x,y,d):', 258, 283, 794)
event:1, (x,y): (408,285)


('p2d(x,y,d):', 408, 285, 767)
('points2d:', [[258, 283, 794], [408, 285, 767]])

In [27]:
camera = calibs[0].camera
camera.zNear = 400
points3d = get3dPoints(points2d,camera,depth)
print('points3d:', points3d)

dist = []
if len(points3d) > 1:
    for i in range(len(points3d)):
        if i>0:
            dist.append(np.linalg.norm(points3d[i]-points3d[i-1]))
print('dist:',dist)

('3d point:', array([ -84.9209671 ,   59.86231995,  794.00006104]))
('3d point:', array([ 119.68782806,   60.51630402,  767.        ]))
('points3d:', [array([ -84.9209671 ,   59.86231995,  794.00006104]), array([ 119.68782806,   60.51630402,  767.        ])])
('dist:', [206.38359927657595])


In [34]:
### Correct values for kinect1/xtion rgb and registered depth calibration.
points3db = []
fx = 525.0
fy = 525.0
cx = 319.5
cy = 239.5

u1 = points2d[0][0]
v1 = points2d[0][1]
d1 = points2d[0][2]
u2 = points2d[1][0]
v2 = points2d[1][1]
d2 = points2d[1][2]

z1 = d1 #/ 1000.0
x1 = (u1 - cx) * z1 / fx
y1= (v1 - cy) * z1 / fy
z2 = d2 #/ 1000.0
x2 = (u2 - cx) * z2 / fx
y2= (v2 - cy) * z2 / fy
p3d1 = np.array([x1, y1, z1])
points3db.append(p3d1)
p3d2 = np.array([x2, y2, z2])
points3db.append(p3d2)
print('points3d:', points3db)
np.linalg.norm(p3d2-p3d1)

('points3d:', [array([ -93.01142857,   65.78857143,  794.        ]), array([ 129.29428571,   66.47333333,  767.        ])])


223.94039274536391

In [33]:
print(calibs[0].camera,calibs[1].camera)

({'orientation': [0.0, 0.0, 0.0, 1.0], 'zFar': 10000.0, 'nearPlaneSize': [448.85333251953125, -336.6399841308594], 'perspective': True, 'position': [0.0, 0.0, 0.0], 'zNear': 400.0, 'center': [0.0, 0.0]}, {'orientation': [0.0, 0.0, 0.0, 1.0], 'zFar': 10000.0, 'nearPlaneSize': [448.85333251953125, -336.6399841308594], 'perspective': True, 'position': [0.0, 0.0, 0.0], 'zNear': 400.0, 'center': [0.0, 0.0]})


In [30]:
449./400.

1.1225